In [ ]:
knitr::opts_chunk$set(echo = TRUE)

For this tutorial, we will be analyzing a single-cell ATAC-seq dataset of human
peripheral blood mononuclear cells (PBMCs) provided by 10x Genomics. The
following files are used in this vignette, all available through the 10x
Genomics website:

* The [Raw data](https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5)  
* The [Metadata](https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_singlecell.csv)  
* The [fragments file](https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz)
* The fragments file [index](https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz.tbi)

<details>
  <summary>**View data download code**</summary>

To download all the required files, you can run the following lines in a shell:

In [ ]:
%%sh
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_singlecell.csv
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz.tbi

</details>

First load in Signac, Seurat, and some other packages we will be using for
analyzing human data.

In [ ]:
library(Signac)
library(Seurat)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
# library(ggplot2)
library(patchwork)
set.seed(1234)

In [ ]:
BiocManager::install("biovizBase")


## Pre-processing workflow

When pre-processing chromatin data, Signac uses information from two related
input files, both of which can be created using CellRanger:
  
  * **Peak/Cell matrix**. This is analogous to the gene expression count matrix used to analyze single-cell RNA-seq. However, instead of genes, each row of the matrix represents a region of the genome (a peak), that is predicted to represent a region of open chromatin. Each value in the matrix represents the number of Tn5 integration sites for each single barcode (i.e. a cell) that map within each peak. You can find more detail on the [10X Website](https://support.10xgenomics.com/single-cell-atac/software/pipelines/latest/output/matrices).

  * **Fragment file**. This represents a full list of all unique fragments across all single cells. It is a substantially larger file, is slower to work with, and is stored on-disk (instead of in memory). However, the advantage of retaining this file is that it contains all fragments associated with each single cell, as opposed to only fragments that map to peaks. More information about the fragment file can be found on the 10x Genomics [website](https://support.10xgenomics.com/single-cell-atac/software/pipelines/latest/output/fragments) or on the [sinto website](https://timoast.github.io/sinto/basic_usage.html#create-scatac-seq-fragments-file).

We start by creating a Seurat object using the peak/cell matrix and cell
metadata generated by `cellranger-atac`, and store the path to the fragment
file on disk in the Seurat object:

In [ ]:


counts <- Read10X_h5(filename = "C:\\Users\\ssrikrishnan6\\scATAC\\Signac\\vignette_disjoint\\atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5")

metadata <- read.csv(
  file = "C:\\Users\\ssrikrishnan6\\scATAC\\Signac\\vignette_disjoint\\atac_v1_pbmc_10k_singlecell.csv",
  header = TRUE,
  row.names = 1
)

chrom_assay <- CreateChromatinAssay(
  counts = counts,
  sep = c(":", "-"),
  genome = 'hg19',
  fragments = 'C:\\Users\\ssrikrishnan6\\scATAC\\Signac\\vignette_disjoint\\atac_v1_pbmc_10k_fragments.tsv.gz',
  min.cells = 10,
  min.features = 200
)

pbmc <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks",
  meta.data = metadata
)

In [ ]:
pbmc

The ATAC-seq data is stored using a custom assay, the `ChromatinAssay`. This
enables some specialized functions for analysing genomic single-cell assays such
as scATAC-seq. By printing the assay we can see some of the additional
information that can be contained in the `ChromatinAssay`, including motif
information, gene annotations, and genome information.

In [ ]:
pbmc[['peaks']]

For example, we can call `granges` on a Seurat object with a `ChromatinAssay`
set as the active assay (or on a `ChromatinAssay`) to see the genomic ranges
associated with each feature in the object. See the
[object interaction vignette](data_structures.html) for more information
about the `ChromatinAssay` class.

In [ ]:
granges(pbmc)

We can also add gene annotations to the `pbmc` object for the human genome.
This will allow downstream functions to pull the gene annotation information
directly from the object.

In [ ]:
# extract gene annotations from EnsDb
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v75)

# change to UCSC style since the data was mapped to hg19
seqlevelsStyle(annotations) <- 'UCSC'

# add the gene information to the object
Annotation(pbmc) <- annotations

## Computing QC Metrics

We can now compute some QC metrics for the scATAC-seq experiment. We currently
suggest the following metrics below to assess data quality. As with scRNA-seq,
the expected range of values for these parameters will vary depending on your
biological system, cell viability, and other factors.

* Nucleosome banding pattern: The histogram of DNA fragment sizes (determined from the paired-end sequencing reads) should exhibit a strong nucleosome banding pattern corresponding to the length of DNA wrapped around a single nucleosome. We calculate this per single cell, and quantify the approximate ratio of mononucleosomal to nucleosome-free fragments (stored as `nucleosome_signal`)

* Transcriptional start site (TSS) enrichment score. The [ENCODE project](https://www.encodeproject.org/) has defined an ATAC-seq targeting score based on the ratio of fragments centered at the TSS to fragments in TSS-flanking regions (see https://www.encodeproject.org/data-standards/terms/). Poor ATAC-seq experiments typically will have a low TSS enrichment score. We can compute this metric for each cell with the `TSSEnrichment()` function, and the results are stored in metadata under the column name `TSS.enrichment`.

* Total number of fragments in peaks: A measure of cellular sequencing depth / complexity. Cells with very few reads may need to be excluded due to low sequencing depth. Cells with extremely high levels may represent doublets, nuclei clumps, or other artefacts.

* Fraction of fragments in peaks: Represents the fraction of all fragments that fall within ATAC-seq peaks. Cells with low values (i.e. <15-20%) often represent low-quality cells or technical artifacts that should be removed. Note that this value can be sensitive to the set of peaks used.

* Ratio reads in genomic blacklist regions The [ENCODE project](https://www.encodeproject.org/) has provided a list of [blacklist regions](https://github.com/Boyle-Lab/Blacklist), representing reads which are often associated with artefactual signal. Cells with a high proportion of reads mapping to these areas (compared to reads mapping to peaks) often represent technical artifacts and should be removed. ENCODE blacklist regions for human (hg19 and GRCh38), mouse (mm10), Drosophila (dm3), and C. elegans (ce10) are included in the Signac package.

Note that the last three metrics can be obtained from the output of CellRanger
(which is stored in the object metadata), but can also be calculated for non-10x
datasets using Signac (more information at the end of this document).

In [ ]:
# compute nucleosome signal score per cell
pbmc <- NucleosomeSignal(object = pbmc)

# compute TSS enrichment score per cell
pbmc <- TSSEnrichment(object = pbmc, fast = FALSE)

# add blacklist ratio and fraction of reads in peaks
pbmc$pct_reads_in_peaks <- pbmc$peak_region_fragments / pbmc$passed_filters * 100
pbmc$blacklist_ratio <- pbmc$blacklist_region_fragments / pbmc$peak_region_fragments

We can inspect the TSS enrichment scores by grouping the cells based on the
score and plotting the accessibility signal over all TSS sites. Setting the 
`fast=TRUE` option in `TSSEnrichment()` will only compute the TSS enrichment
score without storing the entire cell by position matrix of Tn5 insertion
frequency for each cell, and can save memory. However, setting `fast=TRUE` will
not allow downstream plotting of the TSS enrichment signal for different groups
of cells using the `TSSPlot()` function, shown here:

In [ ]:
pbmc$high.tss <- ifelse(pbmc$TSS.enrichment > 2, 'High', 'Low')
TSSPlot(pbmc, group.by = 'high.tss') + NoLegend()

We can also look at the fragment length periodicity for all the cells, and group
by cells with high or low nucleosomal signal strength. You can see that cells
that are outliers for the mononucleosomal / nucleosome-free ratio (based on the
plots above) have different nucleosomal banding patterns. The remaining cells
exhibit a pattern that is typical for a successful ATAC-seq experiment.

In [ ]:
pbmc$nucleosome_group <- ifelse(pbmc$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
FragmentHistogram(object = pbmc, group.by = 'nucleosome_group')

In [ ]:
VlnPlot(
  object = pbmc,
  features = c('pct_reads_in_peaks', 'peak_region_fragments',
               'TSS.enrichment', 'blacklist_ratio', 'nucleosome_signal'),
  pt.size = 0.1,
  ncol = 5
)

Finally we remove cells that are outliers for these QC metrics.

In [ ]:
pbmc <- subset(
  x = pbmc,
  subset = peak_region_fragments > 3000 &
    peak_region_fragments < 20000 &
    pct_reads_in_peaks > 15 &
    blacklist_ratio < 0.05 &
    nucleosome_signal < 4 &
    TSS.enrichment > 2
)
pbmc

## Normalization and linear dimensional reduction

* Normalization: Signac performs term frequency-inverse document frequency (TF-IDF) normalization. This is a two-step normalization procedure, that both normalizes across cells to correct for differences in cellular sequencing depth, and across peaks to give higher values to more rare peaks.

* Feature selection: The low dynamic range of scATAC-seq data makes it challenging to perform variable feature selection, as we do for scRNA-seq. Instead, we can choose to use only the top _n_% of features (peaks) for dimensional reduction, or remove features present in less than _n_ cells with the `FindTopFeatures()` function. Here, we will all features, though we note that we see very similar results when using only a subset of features (try setting min.cutoff to 'q75' to use the top 25% all peaks), with faster runtimes. Features used for dimensional reduction are automatically set as `VariableFeatures()` for the Seurat object by this function.

* Dimension reduction: We next run singular value decomposition (SVD) on the TD-IDF matrix, using the features (peaks) selected above. This returns a reduced dimension representation of the object (for users who are more familiar with scRNA-seq, you can think of this as analogous to the output of PCA).

The combined steps of TF-IDF followed by SVD are known as latent semantic
indexing (LSI), and were first introduced for the analysis of scATAC-seq data by
[Cusanovich et al. 2015](https://science.sciencemag.org/content/367/6473/45.full).

In [ ]:
pbmc <- RunTFIDF(pbmc)
pbmc <- FindTopFeatures(pbmc, min.cutoff = 'q0')
pbmc <- RunSVD(pbmc)

The first LSI component often captures sequencing depth (technical variation)
rather than biological variation. If this is the case, the component should be
removed from downstream analysis. We can assess the correlation between each LSI
component and sequencing depth using the `DepthCor()` function:

In [ ]:
DepthCor(pbmc)

Here we see there is a very strong correlation between the first LSI component
and the total number of counts for the cell, so we will perform downstream steps
without this component.

## Non-linear dimension reduction and clustering

Now that the cells are embedded in a low-dimensional space, we can use methods
commonly applied for the analysis of scRNA-seq data to perform graph-based
clustering and non-linear dimension reduction for visualization. The functions
`RunUMAP()`, `FindNeighbors()`, and `FindClusters()` all come from the Seurat
package.

In [ ]:
pbmc <- RunUMAP(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindNeighbors(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindClusters(object = pbmc, verbose = FALSE, algorithm = 3)
DimPlot(object = pbmc, label = TRUE) + NoLegend()

## Create a gene activity matrix

The UMAP visualization reveals the presence of multiple cell groups in human 
blood. If you are familiar with our
[scRNA-seq analyses of PBMC](https://satijalab.org/seurat/v3.0/pbmc3k_tutorial.html),
you may even recognize the presence of certain myeloid and lymphoid populations
in the scATAC-seq data. However, annotating and interpreting clusters is more
challenging in scATAC-seq data as much less is known about the functional roles
of noncoding genomic regions than is known about protein coding regions (genes). 

However, we can try to quantify the activity of each gene in the genome by
assessing the chromatin accessibility associated with each gene, and create a
new gene activity assay derived from the scATAC-seq data. Here we will use a
simple approach of summing the fragments intersecting the gene body and promoter
region (we also recommend exploring the
[Cicero](https://cole-trapnell-lab.github.io/cicero-release/) tool, which can
accomplish a similar goal, and we provide a vignette showing how to run Cicero
within a Signac workflow [here](cicero.html)). 

To create a gene activity matrix, we extract gene coordinates and extend them
to include the 2 kb upstream region (as promoter accessibility is often
correlated with gene expression). We then count the number of fragments for each
cell that map to each of these regions, using the using the `FeatureMatrix()`
function. These steps are automatically performed by the `GeneActivity()`
function:

In [ ]:
gene.activities <- GeneActivity(pbmc)

In [ ]:
# add the gene activity matrix to the Seurat object as a new assay and normalize it
pbmc[['RNA']] <- CreateAssayObject(counts = gene.activities)
pbmc <- NormalizeData(
  object = pbmc,
  assay = 'RNA',
  normalization.method = 'LogNormalize',
  scale.factor = median(pbmc$nCount_RNA)
)

Now we can visualize the activities of canonical marker genes to help interpret
our ATAC-seq clusters. Note that the activities will be much noisier than
scRNA-seq measurements. This is because they represent measurements from sparse
chromatin data, and because they assume a general correspondence between gene
body/promoter accessibility and gene expression which may not always be the
case. Nonetheless, we can begin to discern populations of monocytes, B, T, and
NK cells based on these gene activity profiles. However, further subdivision of
these cell types is challenging based on supervised analysis alone.

In [ ]:
DefaultAssay(pbmc) <- 'RNA'

FeaturePlot(
  object = pbmc,
  features = c('MS4A1', 'CD3D', 'LEF1', 'NKG7', 'TREM1', 'LYZ'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 3
)

## Integrating with scRNA-seq data

To help interpret the scATAC-seq data, we can classify cells based on an
scRNA-seq experiment from the same biological system (human PBMC). We utilize
methods for cross-modality integration and label transfer, described 
[here](https://doi.org/10.1016/j.cell.2019.05.031), with a more in-depth
tutorial [here](https://satijalab.org/seurat/v3.0/atacseq_integration_vignette.html).
We aim to identify shared correlation patterns in the gene activity matrix and
scRNA-seq dataset to identify matched biological states across the two
modalities. This procedure returns a classification score for each cell for each
scRNA-seq-defined cluster label.

Here we load a pre-processed scRNA-seq dataset for human PBMCs, also provided by
10x Genomics. You can download the raw data for this experiment from the 10x 
[website](https://support.10xgenomics.com/single-cell-gene-expression/datasets/3.0.0/pbmc_10k_v3),
and view the code used to construct this object on
[GitHub](https://github.com/satijalab/Integration2019/blob/master/preprocessing_scripts/pbmc_10k_v3.R).
Alternatively, you can download the pre-processed Seurat object 
[here](https://signac-objects.s3.amazonaws.com/pbmc_10k_v3.rds).

In [ ]:
# Load the pre-processed scRNA-seq data for PBMCs
pbmc_rna <- readRDS("C:\\Users\\ssrikrishnan6\\scATAC\\Signac\\vignette_disjoint\\pbmc_10k_v3.rds")

transfer.anchors <- FindTransferAnchors(
  reference = pbmc_rna,
  query = pbmc,
  reduction = 'cca'
)

predicted.labels <- TransferData(
  anchorset = transfer.anchors,
  refdata = pbmc_rna$celltype,
  weight.reduction = pbmc[['lsi']],
  dims = 2:30
)

pbmc <- AddMetaData(object = pbmc, metadata = predicted.labels)

In [ ]:
library(ggplot2)

In [ ]:
plot1 <- DimPlot(
  object = pbmc_rna,
  group.by = 'celltype',
  label = TRUE,
  repel = TRUE) + NoLegend() + ggtitle('scRNA-seq')

plot2 <- DimPlot(
  object = pbmc,
  group.by = 'predicted.id',
  label = TRUE,
  repel = TRUE) + NoLegend() + ggtitle('scATAC-seq')

plot1 + plot2

You can see that the scRNA-based classifications are entirely consistent with
the UMAP visualization, computed only on the scATAC-seq data. We can now easily
annotate our scATAC-seq-derived clusters (alternatively, we could use the RNA
classifications themselves). We note that cluster 14 maps to CD4 Memory T cells,
but is a very small cluster with lower QC metrics. As this group is likely
representing low-quality cells, we remove it from downstream analysis. 

In [ ]:
pbmc <- subset(pbmc, idents = 14, invert = TRUE)
pbmc <- RenameIdents(
  object = pbmc,
  '0' = 'CD14 Mono',
  '1' = 'CD4 Memory',
  '2' = 'CD8 Effector',
  '3' = 'CD4 Naive',
  '4' = 'CD14 Mono',
  '5' = 'DN T',
  '6' = 'CD8 Naive',
  '7' = 'NK CD56Dim',
  '8' = 'pre-B',
  '9' = 'CD16 Mono',
  '10' = 'pro-B',
  '11' = 'DC',
  '12' = 'NK CD56bright',
  '13' = 'pDC'
)

## Find differentially accessible peaks between clusters

To find differentially accessible regions between clusters of cells, we can
perform a differential accessibility (DA) test. We utilize logistic regression
for DA, as suggested by
[Ntranos et al. 2018](https://www.biorxiv.org/content/10.1101/258566v2)
for scRNA-seq data, and add the total number of fragments as a latent variable
to mitigate the effect of differential sequencing depth on the result.
For sparse data (such as scATAC-seq), we find it is often necessary
to lower the `min.pct` threshold in `FindMarkers()` from the default (0.1, which
was designed for scRNA-seq data). Here we
will focus on comparing Naive CD4 cells and CD14 monocytes, but any groups of
cells can be compared using these methods. We can also visualize these marker
peaks on a violin plot, feature plot, dot plot, heat map, or any
[visualization tool in Seurat](https://satijalab.org/seurat/v3.0/visualization_vignette.html).

In [ ]:
# change back to working with peaks instead of gene activities
DefaultAssay(pbmc) <- 'peaks'

da_peaks <- FindMarkers(
  object = pbmc,
  ident.1 = "CD4 Naive",
  ident.2 = "CD14 Mono",
  test.use = 'LR',
  latent.vars = 'peak_region_fragments'
)

head(da_peaks)

In [ ]:
plot1 <- VlnPlot(
  object = pbmc,
  features = rownames(da_peaks)[1],
  pt.size = 0.1,
  idents = c("CD4 Naive","CD14 Mono")
)
plot2 <- FeaturePlot(
  object = pbmc,
  features = rownames(da_peaks)[1],
  pt.size = 0.1
)

plot1 | plot2

Another way to find DA regions between two groups of cells is to look at the
fold change accessibility between two groups of cells. This can be much faster
than running more sophisticated DA tests, but is not able to account for
latent variables such as differences in the total sequencing depth between
cells, and does not perform any statistical test. However, this can still be a
useful way to quickly explore data, and can be performed using the `FoldChange()`
function in Seurat.

In [ ]:
fc <- FoldChange(pbmc, ident.1 = "CD4 Naive", ident.2 = "CD14 Mono")
head(fc)

Peak coordinates can be difficult to interpret alone. We can find the closest
gene to each of these peaks using the `ClosestFeature()` function. If you
explore the gene lists, you will see that peaks open in Naive T cells are close
to genes such as *BCL11B* and *GATA3* (key regulators of  T cell differentiation
), while peaks open in monocytes are close to genes such as *CEBPB* (a key 
regulator of monocyte differentiation). We could follow up this result further
by doing gene ontology enrichment analysis on the gene sets returned by
`ClosestFeature()`, and there are many R packages that can do this (see the
[`GOstats`](https://bioconductor.org/packages/release/bioc/html/GOstats.html)
package for example).

In [ ]:
open_cd4naive <- rownames(da_peaks[da_peaks$avg_log2FC > 3, ])
open_cd14mono <- rownames(da_peaks[da_peaks$avg_log2FC < -3, ])

closest_genes_cd4naive <- ClosestFeature(pbmc, regions = open_cd4naive)
closest_genes_cd14mono <- ClosestFeature(pbmc, regions = open_cd14mono)

In [ ]:
head(closest_genes_cd4naive)

In [ ]:
head(closest_genes_cd14mono)

## Plotting genomic regions

We can plot the frequency of Tn5 integration across regions of the genome for
cells grouped by cluster, cell type, or any other metadata stored in the object
for any genomic region using the `CoveragePlot()` function.
These represent pseudo-bulk accessibility tracks, where signal from
all cells within a group have been averaged together to visualize the DNA 
accessibility in a region (thanks to Andrew Hill for giving the inspiration for
this function in his excellent [blog post](http://andrewjohnhill.com/blog/2019/04/12/streamlining-scatac-seq-visualization-and-analysis/)). Alongside these accessibility tracks we can visualize other
important information including gene annotation, peak coordinates, and genomic
links (if they're stored in the object). See the
[visualization vignette](visualization.html) for more information.

In [ ]:
# set plotting order
levels(pbmc) <- c("CD4 Naive","CD4 Memory","CD8 Naive","CD8 Effector","DN T","NK CD56bright","NK CD56Dim","pre-B",'pro-B',"pDC","DC","CD14 Mono",'CD16 Mono')

CoveragePlot(
  object = pbmc,
  region = rownames(da_peaks)[1],
  extend.upstream = 40000,
  extend.downstream = 20000
)

We can also create an interactive version of these plots using the
`CoverageBrowser()` function. Here is a recorded demonstration showing how we
can use `CoverageBrowser()` to browser the genome and adjust plotting parameters
interactively. The "Save plot" button in `CoverageBrowser()` will add the
current plot to a list of `ggplot` objects that is returned when the browser
session is ended by pressing the "Done" button, allowing interesting views
to be saved during an interactive session.

<iframe width="560" height="315" src="https://www.youtube.com/embed/S9b5rN32IC8" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

<details>
  <summary>**Working with datasets that were not quantified using CellRanger**</summary>

The CellRanger software from 10x Genomics generates several useful QC metrics
per-cell, as well as a peak/cell matrix and an indexed fragments file. In the
above vignette, we utilize the CellRanger outputs, but provide alternative
functions in Signac for many of the same purposes here.

### Generating a peak/cell or bin/cell matrix

The `FeatureMatrix` function can be used to generate a count matrix containing
any set of genomic ranges in its rows. These regions could be a set of peaks, or
bins that span the entire genome.

In [ ]:
# not run
# peak_ranges should be a set of genomic ranges spanning the set of peaks to be quantified per cell
peak_matrix <- FeatureMatrix(
  fragments = Fragments(pbmc),
  features = peak_ranges
)

For convenience, we also include a `GenomeBinMatrix()` function that will
generate a set of genomic ranges spanning the entire genome for you, and run
`FeatureMatrix()` internally to produce a genome bin/cell matrix.

In [ ]:
# not run
bin_matrix <- GenomeBinMatrix(
  fragments = Fragments(pbmc),
  genome = seqlengths(pbmc),
  binsize = 5000
)

### Counting fraction of reads in peaks

The function `FRiP()` will count the fraction of reads in peaks for each cell,
given a peak/cell assay and a bin/cell assay. Note that this can be run on a
subset of the genome, so that a bin/cell assay does not need to be computed for
the whole genome. This will return a Seurat object will metadata added
corresponding to the fraction of reads in peaks for each cell.

In [ ]:
# not run
total_fragments <- CountFragments("'../vignette_data/atac_v1_pbmc_10k_fragments.tsv.gz'")
pbmc$fragments <- total_fragments[colnames(pbmc), "frequency_count"]

pbmc <- FRiP(
  object = pbmc,
  assay = 'peaks',
  total.fragments = 'fragments'
)

### Counting fragments in genome blacklist regions

The ratio of reads in genomic blacklist regions, that are known to artifactually
accumulate reads in genome sequencing assays, can be diagnostic of low-quality
cells. We provide blacklist region coordinates for several genomes (hg19, hg38,
mm9, mm10, ce10, ce11, dm3, dm6) in the Signac package for convenience. These
regions were provided by the ENCODE consortium, and we encourage users to cite
their [paper](https://doi.org/10.1038/s41598-019-45839-z) if you use the regions
in your analysis. The `FractionCountsInRegion()` function can be used to
calculate the fraction of all counts within a given set of regions per cell.
We can use this function and the blacklist regions to find the fraction of
blacklist counts per cell.

In [ ]:
# not run
pbmc$blacklist_fraction <- FractionCountsInRegion(
  object = pbmc, 
  assay = 'peaks',
  regions = blacklist_hg19
)

In [ ]:
saveRDS(object = pbmc, file = "../vignette_data/pbmc.rds")

</details>

<details>
  <summary>**Session Info**</summary>

In [ ]:
sessionInfo()

</details>